In [161]:
import csv
import re
import os
import pandas as pd
import numpy as np
import xlsxwriter
import shutil

from datetime import datetime, date

In [162]:
file_root = './files'
church_export_root = './momentum/church'
event_export_root = './momentum/events'

pricing_breakdown = {
    '10-17-24': 80,
    '10-31-24': 125,
    '11-08-24': 165
}

event_exclusion_data_cols = [
    'Submission Date',
    "Flow Status",
    'Form Submission Date',
    'District',
    'I am registering as a',
    'Are you Participating or Spectating?',
    'First Name.1',
    'Last Name.1',
    'Birthday',
    'Street Address',
    'Street Address Line 2',
    'City',
    'State / Province',
    'Postal / Zip Code',
    'Cell Phone',
    'Your Email',
    'Gender',
    'Shirt Size',
    'What is your youth leader/pastor\'s email?',
    'First Name.2',
    'Last Name.2',
    'Parent/Guardian Email',
    'Parent Cell Phone',
    'Please list any medical problems',
    'Please list any allergies',
    'Please list past surgeries',
    'Please list any medications and dosage you will be taking',
    'First Name.3',
    'Last Name.3',
    'Home Phone',
    'Work Phone',
    'Contact Phone',
    'Insurance Company',
    'Policy Number #',
    'Do you plan on attending TNT@TNU April 13th-April 16th?',
    'Payment',
    'Intenal Notes'
]

write_file_exlusion = ['First Name', 'Last Name', 'Grade Level', 'Age Level Individual Sport', 'Age Level', 'What church are you a part of?']

event_categories = [
    'Art',
    'Creative Ministries',
    'Creative Writing',
    'Speech',
    'Academics',
    'Vocal Music',
    'Instrumental Music',
    'Individual Sports',
    'Team Sports',
    "Quizzing (TNT Only) "
]

church_list = [
   "Ashland City Faith Community",
    "Bent Knee Cowboy",
    "Betel",
    "Bethel (Nashville First)",
    "Bridgepoint",
    "Ciudad de Refugio",
    "Clarksdale Grace Community",
    "Clarksville First",
    "Clarksville Grace",
    "Clarksville Hope Riders Biker",
    "Clarksville Korean Body of Christ",
    "Clarksville Park Lane",
    "Cleveland First",
    "Columbia First",
    "Columbia Grace",
    "Concord Community",
    "Covington",
    "Crossroads (MS)",
    "Crossroads Community",
    "DaySpring Community",
    "Dover First",
    "Eben-Ezer",
    "Elmore",
    "Erin",
    "Fellowship",
    "Fly",
    "Foundry",
    "Franklin Community of Faith",
    "Fulton",
    "Gallatin",
    "Gloster",
    "Gulf Coast Family",
    "Hispana Betel de Gallatin",
    "Gateway Community",
    "Goodlettsville",
    "Gray's Chapel",
    "Griffin's Chapel",
    "Harmony",
    "Hattiesburg First",
    "Hazlehurst",
    "Hendersonville",
    "Hermitage",
    "Hillside Community",
    "Hope Springs",
    "Betel Hermitage",
    "Highland",
    "Jason Chapel",
    "Joelton",
    "Jones Chapel",
    "Kingston Springs",
    "Laurel Bresee",
    "Laurel First",
    "Lewisburg First",
    "Libertad en Cristo",
    "Long Creek",
    "Madison",
    "Madison Hispanic",
    "Magnolia",
    "Memphis Calvary",
    "Memphis Eastside (Hispanic)",
    "Memphis Emmanuel",
    "Memphis Friendship",
    "Memphis Grace",
    "Memphis Holiness Tabernacle",
    "Memphis New Hope",
    "Memphis Refreshing Springs",
    "Meridian Fitkins Memorial",
    "Meridian Northside",
    "Millington",
    "Mt. Wesley-Akin Chapel",
    "Music City West",
    "Nashville Bell Road",
    "Nashville Christ",
    "Nashville Collective",
    "Nashville Community Fellowship",
    "Nashville Donelson",
    "Nashville First (Main Campus)",
    "Nashville First Haitian",
    "Nashville Glencliff Hispanic",
    "Nashville Grace",
    "Nashville Inglewood",
    "Nashville Trevecca Community",
    "Nashville Westwind Community of Faith",
    "New Vision",
    "Nueva Esperanza",
    "Old Hickory",
    "Oxford",
    "Palabra de Vida",
    "Paris First",
    "Pearson Chapel",
    "Pine Hill",
    "Portland",
    "Quitman",
    "Rosebloom",
    "Savannah",
    "Senatobia Nabors Memorial",
    "Spring Hill Restoration Community",
    "Starkville Wesley Community",
    "The House",
    "The Neighborhood Church",
    "Upper Room Fellowship",
    "Victory Praise & Worship",
    "Waverly",
    "Way of the Cross",
    "White House",
    "Woodlawn Community",
    "Not Representing a Church"
]

In [163]:
files = os.listdir(file_root)
files = [f for f in files if os.path.isfile(f'{file_root}/{f}') and f != '.DS_Store']
files

['Momentum_Registration_-_20242024-11-08_14_39_44.csv']

In [164]:
def create_directory(process):
    outdir = f'./momentum/{process}/{date.today()}/'

    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [165]:
def write_csv(df, df_name, process, filename):
    create_directory(process)
    if df_name not in write_file_exlusion:
        df.to_csv(f'{filename}.csv')

In [166]:
def write_excel(df, df_name, process, filename, indexed=False):
    print(filename)
    create_directory(process)
    if df_name not in write_file_exlusion:
        df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [167]:
def move_to_processed(filename):
    src_file = f'{file_root}/{filename}'
    dst_file = f'./momentum/processed/{filename}'

    shutil.move(src_file, dst_file)

In [168]:
def strip_filename_from_download_date(filename):
    filename_parts = filename.split('_-_')
    return filename_parts[0]

strip_filename_from_download_date(files[0])

'Momentum_Registration'

In [169]:
def get_price(row):
    submission_date_string = row['Form Submission Date']
    price_dates = pricing_breakdown.keys()
    if row['I am registering as a'] == 'Adult':
        return 35

    if row['Are you Participating or Spectating?'] == 'Spectator (non-competing participant)':
        return 50

    for price_date_string in price_dates:
        submission_date = datetime.strptime(submission_date_string, '%b %d, %Y')
        price_date = datetime.strptime(price_date_string, '%m-%d-%y')
        if submission_date <= price_date:
            return pricing_breakdown[price_date_string]

In [170]:
def create_student_categories(file, filename):
   df_original = pd.read_csv(f'{file_root}/{file}')

   cols = df_original.columns;

   category_cols = [];
   for col in cols:
      if col not in event_exclusion_data_cols:
         category_cols.append(col)

   for category_col in category_cols:
      loc_cols = ['First Name', 'Last Name', 'What church are you a part of?']
      group_by_cols = [category_col]
      if 'sport' in category_col.lower():
         loc_cols.append('Age Level Individual Sport')
      else:
         loc_cols.append('Age Level')

      if category_col == 'Individual Sports':
         group_by_cols.append('Age Level Individual Sport')
      if category_col == 'Team Sports':
         group_by_cols.append('What church are you a part of?')
      if 'Music' in category_col:
         group_by_cols.append('What church are you a part of?')

      df = df_original.copy(deep=True)
      df = df.loc[:, [*loc_cols, category_col]].dropna(subset=[category_col])

      if category_col in event_categories:
         df[category_col] = df.loc[:, category_col].apply(lambda x: x.split('\n'))
         df_dedeup = df.drop_duplicates(subset=loc_cols)
         df_indexed = df_dedeup.explode(category_col).groupby([*group_by_cols, 'First Name', 'Last Name', 'What church are you a part of?']).sum()
         # display(df_indexed)

         export_path = f"{event_export_root}/{date.today()}/{category_col.replace(' ', '_').lower()}"
         write_excel(df_indexed, category_col, 'events', export_path, indexed=True)

# create_student_categories(files[0],strip_filename_from_download_date(files[0]))


In [171]:
def use_xlsx_writer(workbook, df_by_church, church_name, process, filename, indexed=False):
    if "(" in church_name:
        church_name = church_name.split("(")[0]
    
    if len(church_name) >= 31:
        church_name = church_name[:31]

    participants = df_by_church.to_dict()
    
    worksheet = workbook.add_worksheet(f'{church_name}')

    head = [
        'First Name',
        'Last Name',
        'Grade Level',
        'Church',
        'Registration Group',
        'Participation Status',
        'Form Submission Date',
        'Art',
        'Creative Ministries',
        'Creative Writing',
        'Speech',
        'Academics',
        'Vocal Music',
        'Instrumental Music',
        'Individual Sports',
        'Team Sports',
        'Quizzing (TNT Only) ',
        'Transaction',
        'Price',
        'Amount Paid',
        'Total Due'
    ]

    bold = workbook.add_format({'bold': True})

    worksheet.write_row(0, 0, head, bold)
    worksheet.write_column(1, 0, participants["First Name"].values())
    worksheet.write_column(1, 1, participants["Last Name"].values())
    worksheet.write_column(1, 2, participants["Grade Level"].values())
    worksheet.write_column(1, 3, participants["What church are you a part of?"].values()).autofit()
    worksheet.write_column(1, 4, participants["I am registering as a"].values())
    worksheet.write_column(1, 5, participants["Are you Participating or Spectating?"].values())
    worksheet.write_column(1, 6, participants["Form Submission Date"].values())
    worksheet.write_column(1, 7, participants["Art"].values())
    worksheet.write_column(1, 8, participants["Creative Ministries"].values())
    worksheet.write_column(1, 9, participants["Creative Writing"].values())
    worksheet.write_column(1, 10, participants["Speech"].values())
    worksheet.write_column(1, 11, participants["Academics"].values())
    worksheet.write_column(1, 12, participants["Vocal Music"].values())
    worksheet.write_column(1, 13, participants["Instrumental Music"].values())
    worksheet.write_column(1, 14, participants["Individual Sports"].values())
    worksheet.write_column(1, 15, participants["Team Sports"].values())
    worksheet.write_column(1, 16, participants["Quizzing (TNT Only) "].values())
    worksheet.write_column(1, 17, participants["Payment"].values())
    worksheet.write_column(1, 18, participants["Price"].values())
    worksheet.write_column(1, 19, participants["Paid"].values())

    # write totals
    current_row = 1
    participant_count = len(participants["index"])

    # Write empty row for viewing
    worksheet.write_row(participant_count+1, 0, "")
    while current_row <= (participant_count + 1):
        worksheet.write_formula(current_row, 20, f"=SUM(S{current_row + 1} - T{current_row + 1})")
        current_row += 1

    worksheet.merge_range(participant_count + 2, 18, participant_count + 2, 19, 'Total Due at Registration', bold)

    worksheet.write(participant_count + 2, 20, f'=SUM(U2:U{participant_count+1})', bold)
    worksheet.write(participant_count + 3, 16, "Check Number", bold)
    worksheet.write(participant_count + 3, 18, "Check Amount", bold)

   


In [172]:
def create_church_info_sheets(file, filename):
    df_original = pd.read_csv(f'{file_root}/{file}')

    df = df_original.copy(deep=True)
    cols = df_original.columns

    category_cols = []
    for col in cols:
        if col not in event_exclusion_data_cols:
            category_cols.append(col)

    # for category_col in category_cols:
    #     if category_col in event_categories:
    #         df[category_col] = df.loc[:, category_col].str.split('\n')

    cols = ['First Name', 'Last Name', 'Grade Level', 'What church are you a part of?', 'I am registering as a', 'Are you Participating or Spectating?', 'Form Submission Date', *event_categories, 'First Name.1', 'Last Name.1', 'What is your youth leader/pastor\'s email?', 'Payment']
    df = df.loc[:, cols].fillna('')
    df = df.groupby(['What church are you a part of?', 'First Name', 'Last Name'], group_keys=False).apply(lambda x: x)
    # df.drop_duplicates(subset=['First Name', 'Last Name', 'Grade Level'], inplace=True)
    df.sort_values('Last Name')
    df.reset_index(inplace=True)
    df['What church are you a part of?'] = df['What church are you a part of?'].apply(lambda x: x.strip())

    create_directory("registration")
    export_path = f"{church_export_root}/{date.today()}/{date.today()}_registration"
    workbook = xlsxwriter.Workbook(f'{export_path}.xlsx')
    
    for church in church_list:
        df_church = df.loc[df['What church are you a part of?'] == church]
        if not df_church.empty:
            df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
            df_church['Paid'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
            # df_church['Total Due'] = df_church.apply(lambda x: x['Price'] - x['Paid'], axis=1)
            df_church.rename(columns = {"First Name.1": "Youth Leader First Name", "Last Name.1": "Youth Leader Last Name"}, inplace = True)
            # df_church.loc['Total',:] = df_church.sum(axis=0, numeric_only=True)

            # display(df_church)
            # write_excel(df_church, church, 'church', export_path)
            
            use_xlsx_writer(workbook, df_church, church, 'church', export_path)
    
    workbook.close()


# create_church_info_sheets(files[0], strip_filename_from_download_date(files[0]))

In [173]:
for file in files:
    filename = strip_filename_from_download_date(file)

    create_student_categories(file, filename)
    create_church_info_sheets(file, filename)
    move_to_processed(file)

./momentum/events/2024-11-08/quizzing_(tnt_only)_
./momentum/events/2024-11-08/art
./momentum/events/2024-11-08/creative_ministries
./momentum/events/2024-11-08/creative_writing
./momentum/events/2024-11-08/speech
./momentum/events/2024-11-08/academics
./momentum/events/2024-11-08/vocal_music
./momentum/events/2024-11-08/instrumental_music
./momentum/events/2024-11-08/individual_sports
./momentum/events/2024-11-08/team_sports


/var/folders/sh/c2r711c50xldbw4058rxqr5c0000gn/T/ipykernel_5458/3722831837.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['What church are you a part of?', 'First Name', 'Last Name'], group_keys=False).apply(lambda x: x)
/var/folders/sh/c2r711c50xldbw4058rxqr5c0000gn/T/ipykernel_5458/3722831837.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
/var/folders/sh/c2r7